In [287]:
#import the necessary libraries 
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import folium
import statsmodels.api as sm
import scipy.stats
import numpy as np
from math import sqrt
import statsmodels.formula.api as smf

In [288]:
data1 = pd.read_csv('london_flows.csv')
data1= data1[data1.station_origin != 'Battersea Park']
data1 = data1.rename(columns={'station_origin':'Orig','station_destination':'Dest'})
data1 =data1[data1["Orig"] != data1["Dest"]]
data1.head()

,Orig,Dest,flows,population,jobs,distance
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097
1,Abbey Road,Beckton,1,599,442,8510.121774
2,Abbey Road,Blackwall,3,599,665,3775.448872
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220
4,Abbey Road,Canning Town,37,599,15428,2228.923167


In [289]:
#set up the metric calculations
def CalcRSqaured(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2
    
    return R2

def CalcRMSE(observed, estimated):
    """Calculate Root Mean Square Error between a series of observed and estimated values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    res = (observed -estimated)**2
    RMSE = round(sqrt(res.mean()), 3)
    
    return RMSE

## singly constrained



since the population of the origin is unchange/constant.

In [290]:
data1['log_des'] = np.log(data1['jobs']+0.001)

In [291]:
#create the formula (the "-1" indicates no intercept in the regression model).
formula = 'flows ~ Orig+ log_des + distance-1'
#run a production constrained sim
prodsim_exp = smf.glm(formula = formula, data=data1, family=sm.families.Poisson()).fit()
#let's have a look at it's summary
print(prodsim_exp.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                61433
Model:                            GLM   Df Residuals:                    61033
Model Family:                 Poisson   Df Model:                          399
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -9.0994e+05
Date:                Tue, 30 Apr 2024   Deviance:                   1.6477e+06
Time:                        10:35:59   Pearson chi2:                 2.40e+06
No. Iterations:                     8   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Orig[A

In [292]:
prodsim_exp.mu

array([ 77.68538182,   1.4659513 ,   4.12137356, ...,  35.95333931,
       125.1128791 ,  79.69156309])

In [293]:
coefs = pd.DataFrame(prodsim_exp.params)
coefs.reset_index(inplace=True)
coefs.rename(columns = {0:"alpha_i", "index":"coef"}, inplace = True)
to_repl = ["(Orig)", "\[", "\]"]
for x in to_repl:
    coefs["coef"] = coefs["coef"].str.replace(x, "",regex=True)
coefs
data1 = data1.merge(coefs, left_on="Orig", right_on="coef", how = "left")
data1.drop(columns = ["coef"], inplace = True)
data1.head()

,Orig,Dest,flows,population,jobs,distance,log_des,alpha_i
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097,11.271478,-2.914325
1,Abbey Road,Beckton,1,599,442,8510.121774,6.091312,-2.914325
2,Abbey Road,Blackwall,3,599,665,3775.448872,6.499789,-2.914325
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220,10.981421,-2.914325
4,Abbey Road,Canning Town,37,599,15428,2228.923167,9.643939,-2.914325


In [294]:
beta = -prodsim_exp.params[-1]
beta

C:\Users\gg208\AppData\Local\Temp\ipykernel_50780\1130537467.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta = -prodsim_exp.params[-1]


0.00015316618786019308

In [295]:
gamma = prodsim_exp.params[-2]
gamma

C:\Users\gg208\AppData\Local\Temp\ipykernel_50780\1908434405.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  gamma = prodsim_exp.params[-2]


0.7552218741808451

In [296]:
data1["prodsimest1"] = prodsim_exp.mu
data1["prodsimest1"] =data1["prodsimest1"].round()
data1.head(10)

,Orig,Dest,flows,population,jobs,distance,log_des,alpha_i,prodsimest1
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097,11.271478,-2.914325,78.0
1,Abbey Road,Beckton,1,599,442,8510.121774,6.091312,-2.914325,1.0
2,Abbey Road,Blackwall,3,599,665,3775.448872,6.499789,-2.914325,4.0
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220,10.981421,-2.914325,99.0
4,Abbey Road,Canning Town,37,599,15428,2228.923167,9.643939,-2.914325,56.0
5,Abbey Road,Crossharbour,1,599,1208,6686.475560,7.096722,-2.914325,4.0
6,Abbey Road,Custom House,0,599,845,3824.855630,6.739338,-2.914325,5.0
7,Abbey Road,Cutty Sark,2,599,1748,8503.898909,7.466228,-2.914325,4.0
8,Abbey Road,Cyprus,7,599,850,6532.099618,6.745238,-2.914325,3.0
9,Abbey Road,Devons Road,1,599,611,3958.324171,6.415099,-2.914325,4.0


In [297]:
CalcRSqaured(data1["flows"], data1["prodsimest1"])

0.4680715291826876

In [298]:
CalcRMSE(data1["flows"], data1["prodsimest1"])

96.247

In [299]:
formula2 = "flows ~ Orig+ log_des + np.log(distance) -1"
prodsim_pow = smf.glm(formula = formula2, data=data1, family=sm.families.Poisson()).fit()
#let's have a look at it's summary
print(prodsim_pow.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                61433
Model:                            GLM   Df Residuals:                    61033
Model Family:                 Poisson   Df Model:                          399
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.0169e+06
Date:                Tue, 30 Apr 2024   Deviance:                   1.8615e+06
Time:                        10:36:14   Pearson chi2:                 2.78e+06
No. Iterations:                     8   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Orig[A

In [300]:
data1["prodsimest2"] = prodsim_pow.mu

In [301]:
CalcRSqaured(data1["flows"], data1["prodsimest2"])

0.3882757756228749

In [302]:
CalcRMSE(data1["flows"], data1["prodsimest2"])

102.877

## A

In [303]:
data2=data1.copy()


In [304]:
def new_sal(row):
    if row['Dest'] == "Canary Wharf":
        val = row['jobs']/2
    else:
        val = row["jobs"]
    return val
        
data2["jobs2"] = data2.apply(new_sal, axis=1)
data2.head(10)

,Orig,Dest,flows,population,jobs,distance,log_des,alpha_i,prodsimest1,prodsimest2,jobs2
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097,11.271478,-2.914325,78.0,55.010696,78549.0
1,Abbey Road,Beckton,1,599,442,8510.121774,6.091312,-2.914325,1.0,0.986106,442.0
2,Abbey Road,Blackwall,3,599,665,3775.448872,6.499789,-2.914325,4.0,2.755641,665.0
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220,10.981421,-2.914325,99.0,66.457309,29386.0
4,Abbey Road,Canning Town,37,599,15428,2228.923167,9.643939,-2.914325,56.0,49.060962,15428.0
5,Abbey Road,Crossharbour,1,599,1208,6686.475560,7.096722,-2.914325,4.0,2.639417,1208.0
6,Abbey Road,Custom House,0,599,845,3824.855630,6.739338,-2.914325,5.0,3.275120,845.0
7,Abbey Road,Cutty Sark,2,599,1748,8503.898909,7.466228,-2.914325,4.0,2.838947,1748.0
8,Abbey Road,Cyprus,7,599,850,6532.099618,6.745238,-2.914325,3.0,2.056310,850.0
9,Abbey Road,Devons Road,1,599,611,3958.324171,6.415099,-2.914325,4.0,2.476929,611.0


In [305]:
cdatasub = data2.copy()

In [306]:
#calculate some new Dj^gamma and d_ij^beta values
Dj2_gamma = cdatasub["jobs"]**gamma
dist_beta = np.exp(-beta * data2["distance"])
#calcualte the first stage of the Ai values
cdatasub["Ai1"] = Dj2_gamma * dist_beta
#now do the sum over all js bit
A_i = pd.DataFrame(cdatasub.groupby(["Orig"])["Ai1"].agg(np.sum))
#now divide into 1
A_i["Ai1"] = 1/A_i["Ai1"]
A_i.rename(columns={"Ai1":"A_i"}, inplace=True)
#and write the A_i values back into the dataframe
cdatasub = cdatasub.merge(A_i, left_on="Orig", right_index=True, how="left")

#to check everything works, recreate the original estimates
cdatasub["prodsimest3"] = cdatasub["A_i"]*cdatasub['population']*Dj2_gamma*dist_beta
#round
cdatasub["prodsimest3"] = round(cdatasub["prodsimest3"])
cdatasub = cdatasub.drop(columns=['A_i'])
#check
cdatasub[["prodsimest1", "prodsimest3"]]

C:\Users\gg208\AppData\Local\Temp\ipykernel_50780\2765702545.py:7: FutureWarning: The provided callable <function sum at 0x0000017FDAA0A8B0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  A_i = pd.DataFrame(cdatasub.groupby(["Orig"])["Ai1"].agg(np.sum))


,prodsimest1,prodsimest3
0,78.0,78.0
1,1.0,1.0
2,4.0,4.0
3,99.0,99.0
4,56.0,56.0
...,...,...
61428,99.0,99.0
61429,295.0,295.0
61430,36.0,36.0
61431,125.0,125.0


In [307]:
#calculate some new Dj^gamma and d_ij^beta values
Dj3_gamma = cdatasub["jobs2"]**gamma
#calcualte the first stage of the Ai values
cdatasub["Ai1"] = Dj3_gamma * dist_beta
#now do the sum over all js bit
A_i = pd.DataFrame(cdatasub.groupby(["Orig"])["Ai1"].agg(np.sum))
#now divide into 1
A_i["Ai1"] = 1/A_i["Ai1"]
A_i.rename(columns={"Ai1":"A_i2"}, inplace=True)
#and write the A_i values back into the dataframe
cdatasub = cdatasub.merge(A_i, left_on="Orig", right_index=True, how="left")

#to check everything works, recreate the original estimates
cdatasub["prodsimest4"] = cdatasub["A_i2"]*cdatasub["population"]*Dj3_gamma*dist_beta
#round
cdatasub["prodsimest4"] = round(cdatasub["prodsimest4"])
cdatasub = cdatasub.drop(columns=['A_i2'])
cdatasub[["prodsimest1", "prodsimest4"]]

C:\Users\gg208\AppData\Local\Temp\ipykernel_50780\2261764494.py:6: FutureWarning: The provided callable <function sum at 0x0000017FDAA0A8B0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  A_i = pd.DataFrame(cdatasub.groupby(["Orig"])["Ai1"].agg(np.sum))


,prodsimest1,prodsimest4
0,78.0,83.0
1,1.0,2.0
2,4.0,4.0
3,99.0,63.0
4,56.0,60.0
...,...,...
61428,99.0,107.0
61429,295.0,320.0
61430,36.0,39.0
61431,125.0,135.0


## B

In [308]:
cdatasub2 = cdatasub

0.00015316618786019308

In [309]:
beta1=0.0005
beta2=0.0009


In [310]:
dist_beta1 = np.exp(-beta1 * data2["distance"])
#calcualte the first stage of the Ai values
cdatasub["Ai1"] = Dj2_gamma * dist_beta1
#now do the sum over all js bit
A_i = pd.DataFrame(cdatasub.groupby(["Orig"])["Ai1"].agg(np.sum))
#now divide into 1
A_i["Ai1"] = 1/A_i["Ai1"]
A_i.rename(columns={"Ai1":"A_i3"}, inplace=True)
#and write the A_i values back into the dataframe
cdatasub = cdatasub.merge(A_i, left_on="Orig", right_index=True, how="left")

cdatasub["prodsimest5"] = cdatasub["A_i3"]*cdatasub["population"]*Dj2_gamma*dist_beta1
#round
cdatasub = cdatasub.drop(columns=['A_i3'])
cdatasub["prodsimest5"] = round(cdatasub["prodsimest5"])


C:\Users\gg208\AppData\Local\Temp\ipykernel_50780\477399939.py:5: FutureWarning: The provided callable <function sum at 0x0000017FDAA0A8B0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  A_i = pd.DataFrame(cdatasub.groupby(["Orig"])["Ai1"].agg(np.sum))


In [311]:
dist_beta2 = np.exp(-beta2 * data2["distance"])
#calcualte the first stage of the Ai values
cdatasub["Ai1"] = Dj2_gamma * dist_beta2
#now do the sum over all js bit
A_i = pd.DataFrame(cdatasub.groupby(["Orig"])["Ai1"].agg(np.sum))
#now divide into 1
A_i["Ai1"] = 1/A_i["Ai1"]
A_i.rename(columns={"Ai1":"A_i4"}, inplace=True)
#and write the A_i values back into the dataframe
cdatasub = cdatasub.merge(A_i, left_on="Orig", right_index=True, how="left")

cdatasub["prodsimest6"] = cdatasub["A_i4"]*cdatasub["population"]*Dj2_gamma*dist_beta2
#round
cdatasub = cdatasub.drop(columns=['A_i4'])
cdatasub["prodsimest6"] = round(cdatasub["prodsimest6"])

C:\Users\gg208\AppData\Local\Temp\ipykernel_50780\1830820259.py:5: FutureWarning: The provided callable <function sum at 0x0000017FDAA0A8B0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  A_i = pd.DataFrame(cdatasub.groupby(["Orig"])["Ai1"].agg(np.sum))


In [312]:
cdatasub[["prodsimest1", "prodsimest5","prodsimest6"]]

,prodsimest1,prodsimest5,prodsimest6
0,78.0,12.0,1.0
1,1.0,0.0,0.0
2,4.0,3.0,1.0
3,99.0,45.0,11.0
4,56.0,68.0,53.0
...,...,...,...
61428,99.0,16.0,1.0
61429,295.0,243.0,52.0
61430,36.0,22.0,3.0
61431,125.0,328.0,267.0


In [313]:
A=max(cdatasub["prodsimest1"])
B=max(cdatasub["prodsimest5"])
C=max(cdatasub["prodsimest6"])
print(A,B,C)

3686.0 5434.0 9384.0


In [314]:
max_rows_df = pd.concat([
    cdatasub.loc[cdatasub["prodsimest1"].idxmax():cdatasub["prodsimest1"].idxmax()],
    cdatasub.loc[cdatasub["prodsimest5"].idxmax():cdatasub["prodsimest5"].idxmax()],
    cdatasub.loc[cdatasub["prodsimest6"].idxmax():cdatasub["prodsimest6"].idxmax()]
], axis=0)

max_rows_df = max_rows_df[['Orig','Dest','population','jobs','distance',
                           "prodsimest1","prodsimest5","prodsimest6"]]
max_rows_df

,Orig,Dest,population,jobs,distance,prodsimest1,prodsimest5,prodsimest6
55909,Waterloo,Bank and Monument,67372,78549,2542.954444,3686.0,4788.0,4360.0
32594,London Bridge,Bank and Monument,32597,78549,886.728371,2395.0,5434.0,8885.0
18935,Finsbury Park,Highbury & Islington,24735,25385,2057.343960,1048.0,4543.0,9384.0


In [324]:
CW = cdatasub[cdatasub.Dest == 'Canary Wharf']
CW = CW.drop(columns=['log_des','alpha_i','prodsimest3',
                 'prodsimest2','Ai1','jobs2'])
CW = CW.rename(columns={'prodsimest4': 'A'})
CW = CW.rename(columns={'prodsimest5': 'B1'})
CW = CW.rename(columns={'prodsimest6': 'B2'})
CW = CW.rename(columns={'prodsimest1': 'Orignal perdict'})
CW.sample(12)

,Orig,Dest,flows,population,jobs,distance,Orignal perdict,A,B1,B2
1479,Archway,Canary Wharf,51,5642,58772,14463.519883,78.0,47.0,6.0,0.0
6622,Brent Cross,Canary Wharf,25,1494,58772,18538.113322,21.0,13.0,1.0,0.0
53194,Turnpike Lane,Canary Wharf,75,6690,58772,15918.420969,105.0,62.0,8.0,0.0
38056,Oakwood,Canary Wharf,33,1772,58772,23553.720440,28.0,17.0,0.0,0.0
52977,Turnham Green,Canary Wharf,111,3650,58772,18175.623659,37.0,22.0,1.0,0.0
29069,Kew Gardens,Canary Wharf,53,2241,58772,21503.222576,20.0,12.0,0.0,0.0
44518,Seven Sisters,Canary Wharf,173,18780,58772,15324.656059,291.0,173.0,21.0,0.0
24874,High Street Kensington,Canary Wharf,90,2159,58772,13717.834363,24.0,15.0,1.0,0.0
23099,Harrow & Wealdstone,Canary Wharf,13,1741,58772,28471.819856,14.0,8.0,0.0,0.0
30039,Kingsbury,Canary Wharf,108,2727,58772,25060.319552,27.0,16.0,0.0,0.0


In [ ]:
## Compare

## Compare